# Testing of QuTiP QOC on Qiskit using IBM backend

Ben Rosand (brosand@gmail.com)

### Imports

In [ ]:
from src.QOCTester import QOCTester
from qiskit import IBMQ
from qiskit.tools.monitor import job_monitor
import numpy as np

## Setup backend and initialize tester

In [ ]:

IBMQ.load_account()

provider = IBMQ.get_provider(hub='ibm-q-internal', group='deployed', project='default')
backend = provider.get_backend('ibmq_armonk')

In [ ]:
target_name = 'XGate'
subsystem_list = [0]

config = backend.configuration()

tester = QOCTester(config, subsystem_list, p_type='LIN', two_level=True)

### Initialize the target unitary evolution, could be 'x', 'XGate', 'sigmax', 'h', 'hadamard', or 'cnot'

In [ ]:
# tester.make_two_level()
tester.init_unitaries(target_name)


 ### Initialize the time steps to cycle through for the tester

In [ ]:
n_ts_list=np.arange(64,320,64)
tester.time_init(n_ts_list=n_ts_list)

## Run the optimizer and plot the results

In [ ]:
optim = tester.create_pulse_optimizer()

In [ ]:
results = tester.tester_run_optimizer(optim)

In [ ]:
tester.plot_all_results(results)

In [ ]:
# Generate qiskit programs from the resulting amp sequences
programs = tester.gen_qiskit_progs(results, backend)

In [ ]:
# Run the programs on a qiskit backend
jobs = tester.run_qiskit_programs(programs,backend)

In [ ]:
job_monitor(jobs[-1])

In [ ]:
# Create dictionary of qubit 0 excitation
acc = tester.get_acc(jobs)

In [ ]:
# Save the accuracy results to a csv file
tester.write_acc(acc, 'results.csv')

In [ ]:
# If you want to examine a specific schedule here is how to do it
from src.testing_helper import generate_schedule
from qiskit.pulse import DriveChannel
sched = generate_schedule(results[0].final_amps, backend, draw_progs=True)
sched.draw(plot_range=[0,200], channels=[DriveChannel(0)])